In [ ]:
import sqlite3
import pandas as pd
# from anonymity.mondrian import MondrianAnonymizer, is_k_anonymous
# from anonymity.plot import plot_mondrian

### Step 1 - Load the Data

In [3]:
connection = sqlite3.connect('../data.db')
cursor = connection.cursor()
cursor.execute('SELECT SQLITE_VERSION()')
cursor.fetchall()
pd.read_sql('SELECT SQLITE_VERSION()', connection)

,SQLITE_VERSION()
0,3.35.5


In [38]:
personalData = pd.read_csv('../data/personal_data.csv',sep=';')
sensitiveData = pd.read_csv('../data/sensitive_data.csv',sep=';')
personalData.to_sql('personal', connection, if_exists='replace', index=False)
sensitiveData.to_sql('sensitive', connection, if_exists='replace', index=False)

In [37]:
# cursor.execute("DROP TABLE personal")
# cursor.execute("DROP TABLE sensitive")

In [39]:
db_personal = pd.read_sql_query("SELECT * FROM personal", connection)
db_sensitive = pd.read_sql_query("SELECT * FROM sensitive", connection)

In [40]:
db_personal.head()

,nom,prenom,sexe,age,adresse,code_postal
0,BILA,CORINNE,2,60,172 Rue du Pré de l’Eglise 69220 Dracé,69220
1,FRANCOIS,CLAIRE,2,80,35 Avenue des Marronniers 69270 Fontaines-sur-...,69270
2,USTARROZ,YVETTE,2,52,22 Passage de l’Ancienne Mairie 69400 Villefra...,69400
3,FREDET,ANGELIQUE,2,52,47 Avenue du 11 Novembre 1918 69160 Tassin-la-...,69160
4,TEMPLIER,RONAN,1,21,6 Grande Rue 69250 Fleurieu-sur-Saône,69250


In [41]:
db_sensitive.head()

,nom,prenom,sexe,age,adresse,code_postal,date_naissance,consommation_alcool,activite_physique,tension_arterielle
0,ABADIE,LIONEL,1,57,45 Avenue Edouard Herriot 69150 Décines-Charpieu,69150,1963-12-09,élevée,0,élevée
1,ABALO,MORGANE,2,57,6 Allee des Narcisses 69380 Dommartin,69380,1964-02-07,modérée,1,normale
2,ABDEL GADIR,ANNA,2,63,605 Le Bourg 69430 Vernay,69430,1958-04-16,modérée,1,normale
3,ABDESSALEM,IBRAHIM,1,33,1450 Route du Pont Dorieux 69210 Fleurieux-sur...,69210,1988-01-03,modérée,1,normale
4,ABRANTES DUARTE,MAXIME,1,66,25 Rue de Montribloud 69009 Lyon 9e Arrondisse...,69009,1955-05-31,élevée,0,élevée


In [43]:
# Step 1.5
patient_in_join = pd.merge(
    db_personal,
    db_sensitive,
    on=('nom', 'prenom', 'sexe', 'age', 'adresse', 'code_postal')
)
print(len(patient_in_join))
patient_in_join.head()

1002


,nom,prenom,sexe,age,adresse,code_postal,date_naissance,consommation_alcool,activite_physique,tension_arterielle
0,AUPEE,EMELYNE,2,27,6 Rue Paul-Michel Perret 69006 Lyon 6e Arrondi...,69006,1993-10-09,modérée,1,normale
1,DUBESSET,BRUNO,1,65,10 Allée de la Clé des Champs 69580 Sathonay-V...,69580,1956-08-04,modérée,1,élevée
2,JOUNEAU,SOPHIE,2,49,32 Rue Valentin Couturier 69004 Lyon 4e Arrond...,69004,1972-04-03,modérée,1,normale
3,GARAYT,CLÉMENT,1,69,36 Rue Pierre Brunier 69300 Caluire-et-Cuire,69300,1952-03-09,faible,0,très élevée
4,POLEYA,DANIELLE,2,50,10 Montee du Chatel 69440 Saint-André-la-Côte,69440,1971-06-10,modérée,0,très élevée


### Step 2 - Pseudonymization: Replace names

Remove the identifier and replace them with a unique value per patient